In [ ]:
import gzip
import json
import glob

from collections import deque
from elasticsearch import Elasticsearch, helpers


In [ ]:
es = Elasticsearch('localhost:9220')
# es = Elasticsearch('logana01:9220')
bulk_path_pattern = 'data/logana_log*.ndjson.gz'
bulk_size = 5000


In [ ]:
def load_logs(pattern):
    count = 0
    for bulk_path in glob.glob(pattern):
        print(bulk_path)
        with gzip.open(bulk_path, mode='rt', encoding='utf-8') as f:
            action = None
            for line in f:
                json_obj = json.loads(line)
                if action is None:
                    if 'index' in json_obj:
                        action = json_obj.get('index')
                    elif 'create' in json_obj:
                        action = json_obj.get('create')
                else:
                    action['_source'] = json_obj
                    yield action
                    action = None
                    count += 1
                    if count % 10000 == 0:
                        print(f'sent {count} docs')

pb = helpers.parallel_bulk(es, load_logs(bulk_path_pattern), chunk_size=bulk_size, request_timeout=600)
deque(pb, maxlen=0)
